In [1]:
import pandas as pd
import spacy

from collections import defaultdict

In [2]:
df = pd.read_excel("InputTestSet-Reviews48_Ann.xlsx")

In [3]:
df.head()

,UID,PID,Dec,Sent,MComp,Cat,SubCat
0,0,2019_SJf_XhCqKm,Reject,The authors propose to use k-DPP to select a s...,0,NaN,NaN
1,1,2019_SJf_XhCqKm,Reject,"This paper covers the related work nicely, wit...",0,NaN,NaN
2,2,2019_SJf_XhCqKm,Reject,The rest of the paper are also clearly written.,0,NaN,NaN
3,3,2019_SJf_XhCqKm,Reject,"However, I have some concerns about the propos...",0,NaN,NaN
4,4,2019_SJf_XhCqKm,Reject,"- It is not clear how to define the kernel, th...",0,NaN,NaN


In [4]:
gt_dict = {}

for i in range(0, df.shape[0]):
    pid = df.loc[i]["PID"]
    if not pid in gt_dict:
        gt_dict[pid] = {"dec": df.loc[i]["Dec"], "mcomp": set(), "not_mcomp": set()}
    if df.loc[i]["MComp"] == 1:
        gt_dict[pid]["mcomp"].add(df.loc[i]["UID"])
    else:
        gt_dict[pid]["not_mcomp"].add(df.loc[i]["UID"])

In [5]:
stats_dict = {"Accept": [0, 0], "Reject": [0, 0]}

for k, v in gt_dict.items():
    #print(len(v["mcomp"]), len(v["not_mcomp"]), v["dec"])
    stats_dict[v["dec"]][0] += len(v["mcomp"])
    stats_dict[v["dec"]][1] += len(v["not_mcomp"])
    
print(stats_dict)

{'Accept': [48, 644], 'Reject': [69, 744]}


In [6]:
test_set = list(gt_dict.keys())
print("TestSet length: %d\n"%len(test_set), test_set)

TestSet length: 32
 ['2019_SJf_XhCqKm', '2017_Bk0MRI5lg', '2020_SyevYxHtDB', '2018_rJBiunlAW', '2020_rkltE0VKwH', '2018_Hki-ZlbA-', '2019_BJx0sjC5FX', '2020_r1e_FpNFDr', '2020_B1lsXREYvr', '2018_SkZxCk-0Z', '2019_rJzoujRct7', '2018_HkfXMz-Ab', '2017_BJ9fZNqle', '2019_SyxZJn05YX', '2017_B1ckMDqlg', '2017_HJ0NvFzxl', '2017_S1_pAu9xl', '2018_SyYYPdg0-', '2017_BJAA4wKxg', '2019_HyVxPsC9tm', '2019_HylTBhA5tQ', '2019_B1l08oAct7', '2018_H135uzZ0-', '2017_H1oyRlYgg', '2017_r1y1aawlg', '2020_r1eX1yrKwB', '2020_Byg79h4tvB', '2019_H1lFZnR5YX', '2020_BkeWw6VFwr', '2018_HyHmGyZCZ', '2018_HyUNwulC-', '2020_HkgsPhNYPS']


In [7]:
for k in test_set:
    print('{:20}{}'.format(k, gt_dict[k]["mcomp"]))

2019_SJf_XhCqKm     {39, 17, 20, 27, 28, 30}
2017_Bk0MRI5lg      {48, 57}
2020_SyevYxHtDB     {76, 87}
2018_rJBiunlAW      {108, 110, 112, 113, 124, 126}
2020_rkltE0VKwH     {160, 155, 184, 159}
2018_Hki-ZlbA-      {267, 235, 236, 271}
2019_BJx0sjC5FX     {292, 287}
2020_r1e_FpNFDr     {312, 322, 315, 308}
2020_B1lsXREYvr     {376, 401}
2018_SkZxCk-0Z      {449, 443, 445, 486}
2019_rJzoujRct7     {518, 519}
2018_HkfXMz-Ab      {573, 566}
2017_BJ9fZNqle      {627, 623, 615}
2019_SyxZJn05YX     {672, 673, 657, 669, 671}
2017_B1ckMDqlg      {714, 707}
2017_HJ0NvFzxl      {739}
2017_S1_pAu9xl      {792, 809, 810, 806}
2018_SyYYPdg0-      {834, 867, 868, 869, 870, 872, 873, 844, 830}
2017_BJAA4wKxg      {884}
2019_HyVxPsC9tm     {931, 933, 905, 909, 912, 913, 919, 926}
2019_HylTBhA5tQ     {972, 950}
2019_B1l08oAct7     {994, 996, 1064, 1004, 1007, 1044, 1047, 1048, 1055}
2018_H135uzZ0-      {1072, 1079}
2017_H1oyRlYgg      set()
2017_r1y1aawlg      {1125, 1162, 1100, 1102, 1168}
2020_r1eX1y

In [8]:
sents_for_test = defaultdict(list)

for i in range(0, df.shape[0]):
    pid = df.loc[i]["PID"]
    sents_for_test[pid].append((df.loc[i]["UID"], df.loc[i]["Sent"]))

# Basic baseline patterns

In [ ]:
baseline_patterns = ["no comparison with paper", "a reasonable baseline paper is", 
                     "compare the results with paper", "the baseline paper is weak", ""]

In [10]:
nlp = spacy.load('en_core_web_sm')

In [11]:
doc = nlp("Can you compare your results? If possible, could the author provide results on different architecture choices for the stolen model as well as the surrogate model?")

In [71]:
sp_toks = ["result", "method", "task", "dataset", "metric"]

for pid in gt_dict:
    for sent_uid in gt_dict[pid]["mcomp"]:
        doc = nlp(df.loc[sent_uid]["Sent"])
        verb_subtree = []
        
        for s in doc.sents:
            find_special_tokens = {"compar": [], "result": [], "method": [], "task": [], "dataset": [], "metric": []}

            for tok in s:

                if tok.text.lower().startswith("compar"):
                    find_special_tokens["compar"].append(tok)
                else:
                    for k in sp_toks:
                        if tok.text.lower().startswith(k):
                            find_special_tokens[k].append(tok)
                            break
            
            verb_tokens = []
            if find_special_tokens["compar"]:
                for t in find_special_tokens["compar"]:
#                     verb_subtree.append(t.subtree)
                    if t == s.root:
                        simplified_sent = ""
                        for chh in t.lefts:
                            simplified_sent = simplified_sent + " " + chh.text
                        simplified_sent = simplified_sent + " " + t.text
                        for chh in t.rights:
                            simplified_sent = simplified_sent + " " + chh.text
#                         print("SIMP: ", simplified_sent)
                        verb_subtree.append(simplified_sent)
                    else:
                        verb_subtree.append(t.subtree)
            else:
                for k in sp_toks:
                    for i in find_special_tokens[k]:
                        local_vt = []
                        for j in i.ancestors:
                            if j.pos_ == "NOUN":
                                local_vt.append(j)
                        if not local_vt:
                            for j in i.ancestors:
                                if j.pos_ == "VERB":
                                    local_vt.append(j)
                        verb_tokens = verb_tokens + local_vt
                            
                
                for i in verb_tokens:
                    verb_subtree.append(i.subtree)
        
        print("Original Sentence: ", df.loc[sent_uid]["Sent"])
        print("Verb SubTrees: ")
        for i in verb_subtree:
            if type(i) == str:
                print(i)
            else:
                for k in i:
                    print(k, end=" ")
            print("\n")
        print("\n==============================================================================\n")
    break


Original Sentence:  It would be interesting to explore the practicability of the method on more large-scale experiments on image related tasks.
Verb SubTrees: 
the practicability of the method 

more large - scale experiments on image related tasks 



Original Sentence:  I think a clearer emphasis on the novelty, eg: current algorithm with mixing rate analyses or more thorough empirical comparisons will make the paper stronger for resubmission.
Verb SubTrees: 
more thorough empirical comparisons 



Original Sentence:  The authors propose k-DPP as an open loop (oblivious to the evaluation of configurations) method for hyperparameter optimization and provide its empirical study and comparison with other methods such as grid search, uniform random search, low-discrepancy Sobol sequences, BO-TPE (Bayesian optimization using tree-structured Parzen estimator) by Bergstra et al (2011).
Verb SubTrees: 
comparison 



Original Sentence:  Second, their study only applies to a small number like

In [63]:
doc = nlp("Second, their study only applies to a small number like 3-6 hyperparameters with a small k=20) The real challenge lies in scaling up to many hyperparameters or even k-DPP sampling for larger k. Third, the authors do not compare against some relevant, recent work, e.g., Springenberg et al (http://aad.informatik.uni-freiburg.de/papers/16-NIPS-BOHamiANN.pdf) and Snoek et al (https://arxiv.org/pdf/1502.05700.pdf) that is essential for this kind of empirical study.")
for s in doc.sents:
    print(s, end="\n\n")
    find_special_tokens = {"compar": [], "result": [], "method": [], "task": [], "dataset": [], "metric": []}

    for tok in s:

        if tok.text.lower().startswith("compar"):
            find_special_tokens["compar"].append(tok)
        else:
            for k in sp_toks:
                if tok.text.lower().startswith(k):
                    find_special_tokens[k].append(tok)
                    break
    
    verb_tokens = []
    if find_special_tokens["compar"]:
        for t in find_special_tokens["compar"]:
            #verb_subtree.append(t.subtree)
            if t == s.root:
#                 verb_subtree.append(t.children)
#                 print("Alsot he root: ", t)
                simplified_sent = ""
                for chh in t.lefts:
                    simplified_sent = simplified_sent + " " + chh.text
                simplified_sent = simplified_sent + " " + t.text
                for chh in t.rights:
                    simplified_sent = simplified_sent + " " + chh.text
                print(simplified_sent)
            else:
                verb_subtree.append(t.subtree)
    else:
        
        for k in sp_toks:
            for i in find_special_tokens[k]:
                local_vt = []
                for j in i.ancestors:
                    if j.pos_ == "NOUN":
                        local_vt.append(j)
                if not local_vt:
                    for j in i.ancestors:
                        if j.pos_ == "VERB":
                            local_vt.append(j)
                verb_tokens = verb_tokens + local_vt


        for i in verb_tokens:
            verb_subtree.append(i.subtree)
    
    for i in verb_subtree:
        for k in i:
            print(k, end=" ")
        print("\n")
    print("\n==============================================================================\n")


Second, their study only applies to a small number like 3-6 hyperparameters with a small k=20)



















The real challenge lies in scaling up to many hyperparameters or even k-DPP sampling for larger k. Third, the authors do not compare against some relevant, recent work, e.g., Springenberg et al (http://aad.informatik.uni-freiburg.de/papers/16-NIPS-BOHamiANN.pdf)

Alsot he root:  compare
 lies , authors do not compare against et


















and Snoek et al (https://arxiv.org/pdf/1502.05700.pdf) that is essential for this kind of empirical study.





















In [56]:
doc = nlp("It would be interesting to explore the practicability of the methods on more large-scale experiments on image related tasks.")
for tok in doc:
    print(tok.pos_)
#     if tok.text.lower() == "method":
#         for i in tok.ancestors:
#             print(i.text)
#         print("\n================")
#     if tok.text.lower() == "explore":
#         for i in tok.children:
#             print(i.text)
#         print("\n================")

PRON
VERB
AUX
ADJ
PART
VERB
DET
NOUN
ADP
DET
NOUN
ADP
ADV
ADJ
PUNCT
NOUN
NOUN
ADP
NOUN
VERB
NOUN
PUNCT


In [60]:
for i in tok.rights:
    print(i)

In [61]:
dir(tok)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 'ancestors',
 'check_flag',
 'children',
 'cluster',
 'conjuncts',
 'dep',
 'dep_',
 'doc',
 'ent_id',
 'ent_id_',
 'ent_iob',
 'ent_iob_',
 'ent_kb_id',
 'ent_kb_id_',
 'ent_type',
 'ent_type_',
 'get_extension',
 'has_extension',
 'has_vector',
 'head',
 'i',
 'idx',
 'is_alpha',
 'is_ancestor',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_sent_end',
 'is_sent_start',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper',
 'lang',
 'lang_',
 'left_edge',
 'lefts',
 'lemma',
 'lemma_',
 'lex_id',
 'l